In [38]:
# Import
import pandas as pd
import numpy as np
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path

# Allow for reviewing more of the DataFrames
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [39]:
# Read in the CSV files that contain SMA, EMA, and BB/RSI inidcators based on ratio data
googl_df = pd.read_csv("Resources/googl_signals.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
nvda_df = pd.read_csv("Resources/nvda_signals.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
mmm_df = pd.read_csv("Resources/mmm_signals.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)
pg_df = pd.read_csv("Resources/pg_signals.csv", infer_datetime_format=True, index_col="Date", parse_dates=True)

In [40]:
# Concatenate the total value per stock position into a single DataFrame

#SMA 10/100s
sma10_ps_ticker_totals = pd.concat([googl_df['ps_Portfolio Total_sma'], nvda_df['ps_Portfolio Total_sma'], 
                                   mmm_df['ps_Portfolio Total_sma'], pg_df['ps_Portfolio Total_sma']], axis="columns", join="inner")
sma10_ps_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

sma10_pfcf_ticker_totals = pd.concat([googl_df['pfcf_Portfolio Total_sma'], nvda_df['pfcf_Portfolio Total_sma'], 
                                   mmm_df['pfcf_Portfolio Total_sma'], pg_df['pfcf_Portfolio Total_sma']], axis="columns", join="inner")
sma10_pfcf_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

sma10_pe_ticker_totals = pd.concat([googl_df['pe_Portfolio Total_sma'], nvda_df['pe_Portfolio Total_sma'], 
                                   mmm_df['pe_Portfolio Total_sma'], pg_df['pe_Portfolio Total_sma']], axis="columns", join="inner")
sma10_pe_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

#SMA 30/100s
sma30_ps_ticker_totals = pd.concat([googl_df['ps_Portfolio Total_sma30'], nvda_df['ps_Portfolio Total_sma30'],
                                   mmm_df['ps_Portfolio Total_sma30'], pg_df['ps_Portfolio Total_sma30']], axis="columns", join="inner")
sma30_ps_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

sma30_pfcf_ticker_totals = pd.concat([googl_df['pfcf_Portfolio Total_sma30'], nvda_df['pfcf_Portfolio Total_sma30'],
                                   mmm_df['pfcf_Portfolio Total_sma30'], pg_df['pfcf_Portfolio Total_sma30']], axis="columns", join="inner")
sma30_pfcf_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

sma30_pe_ticker_totals = pd.concat([googl_df['pe_Portfolio Total_sma30'], nvda_df['pe_Portfolio Total_sma30'],
                                   mmm_df['pe_Portfolio Total_sma30'], pg_df['pe_Portfolio Total_sma30']], axis="columns", join="inner")
sma30_pe_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

# EMA 10/100s
ema10_ps_ticker_totals = pd.concat([googl_df['ps_Portfolio Total_ema'], nvda_df['ps_Portfolio Total_ema'],
                                   mmm_df['ps_Portfolio Total_ema'], pg_df['ps_Portfolio Total_ema']], axis="columns", join="inner")
ema10_ps_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

ema10_pfcf_ticker_totals = pd.concat([googl_df['pfcf_Portfolio Total_ema'], nvda_df['pfcf_Portfolio Total_ema'],
                                   mmm_df['pfcf_Portfolio Total_ema'], pg_df['pfcf_Portfolio Total_ema']], axis="columns", join="inner")
ema10_pfcf_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

ema10_pe_ticker_totals = pd.concat([googl_df['pe_Portfolio Total_ema'], nvda_df['pe_Portfolio Total_ema'],
                                   mmm_df['pe_Portfolio Total_ema'], pg_df['pe_Portfolio Total_ema']], axis="columns", join="inner")
ema10_pe_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

# EMA 30/100s
ema30_ps_ticker_totals = pd.concat([googl_df['ps_Portfolio Total_ema30'], nvda_df['ps_Portfolio Total_ema30'],
                                   mmm_df['ps_Portfolio Total_ema30'], pg_df['ps_Portfolio Total_ema30']], axis="columns", join="inner")
ema30_ps_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

ema30_pfcf_ticker_totals = pd.concat([googl_df['pfcf_Portfolio Total_ema30'], nvda_df['pfcf_Portfolio Total_ema30'],
                                   mmm_df['pfcf_Portfolio Total_ema30'], pg_df['pfcf_Portfolio Total_ema30']], axis="columns", join="inner")
ema30_pfcf_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']

ema30_pe_ticker_totals = pd.concat([googl_df['pe_Portfolio Total_ema30'], nvda_df['pe_Portfolio Total_ema30'],
                                   mmm_df['pe_Portfolio Total_ema30'], pg_df['pe_Portfolio Total_ema30']], axis="columns", join="inner")
ema30_pe_ticker_totals.columns = ['GOOGL', 'NVDA', 'MMM', 'PG']


In [41]:
# Calculate and plot the sum of the ticker total values for each stock

sma10_ps_ticker_totals['portfolio_total'] = sma10_ps_ticker_totals.sum(axis=1)
sma10_pfcf_ticker_totals['portfolio_total'] = sma10_pfcf_ticker_totals.sum(axis=1)
sma10_pe_ticker_totals['portfolio_total'] = sma10_pe_ticker_totals.sum(axis=1)

sma30_ps_ticker_totals['portfolio_total'] = sma30_ps_ticker_totals.sum(axis=1)
sma30_pfcf_ticker_totals['portfolio_total'] = sma30_pfcf_ticker_totals.sum(axis=1)
sma30_pe_ticker_totals['portfolio_total'] = sma30_pe_ticker_totals.sum(axis=1)

ema10_ps_ticker_totals['portfolio_total'] = ema10_ps_ticker_totals.sum(axis=1)
ema10_pfcf_ticker_totals['portfolio_total'] = ema10_pfcf_ticker_totals.sum(axis=1)
ema10_pe_ticker_totals['portfolio_total'] = ema10_pe_ticker_totals.sum(axis=1)

ema30_ps_ticker_totals['portfolio_total'] = ema30_ps_ticker_totals.sum(axis=1)
ema30_pfcf_ticker_totals['portfolio_total'] = ema30_pfcf_ticker_totals.sum(axis=1)
ema30_pe_ticker_totals['portfolio_total'] = ema30_pe_ticker_totals.sum(axis=1)

In [42]:
# Plot the total portfolio value for each stock per strategy per ratio

# SMA 10/100
plot_sma10_ps_ratios = sma10_ps_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Sales (P/S) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_sma10_pfcf_ratios = sma10_pfcf_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_sma10_pe_ratios = sma10_pe_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Earnings (P/E) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

# SMA 30/100
plot_sma30_ps_ratios = sma30_ps_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Sales (P/S) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_sma30_pfcf_ratios = sma30_pfcf_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_sma30_pe_ratios = sma30_pe_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Earnings (P/E) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

# EMA 10/100
plot_ema10_ps_ratios = ema10_ps_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Sales (P/S) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_ema10_pfcf_ratios = ema10_pfcf_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_ema10_pe_ratios = ema10_pe_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Earnings (P/E) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

# EMA 30/100
plot_ema30_ps_ratios = ema30_ps_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Sales (P/S) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_ema30_pfcf_ratios = ema30_pfcf_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

plot_ema30_pe_ratios = ema30_pe_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Earnings (P/E) Ratio", 
                                  y=['GOOGL', 'NVDA', 'MMM', 'PG'],
                                  ylabel="Total Position Value per Ticker", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True)

In [43]:
# Plot the total portfolio value for all stocks combined per stratgegy per ratio

# SMA 10/100
plot_sma10_ps_price_portfolio = sma10_ps_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Sales (P/S) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_sma10_pfcf_price_portfolio = sma10_pfcf_ticker_totals.hvplot.line(title="Strategy SMA 10/100 Price to Free Cash Flow (P/FCF) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_sma10_pe_price_portfolio = sma10_pe_ticker_totals.hvplot.line(title="Strategy SMA 10/100 on Price to Earnings (P/E) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

# SMA 30/100
plot_sma30_ps_price_portfolio = sma30_ps_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Sales (P/S) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_sma30_pfcf_price_portfolio = sma30_pfcf_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_sma30_pe_price_portfolio = sma30_pe_ticker_totals.hvplot.line(title="Strategy SMA 30/100 on Price to Earnings (P/E) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

# EMA 10/100
plot_ema10_ps_price_portfolio = ema10_ps_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Sales (P/S) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_ema10_pfcf_price_portfolio = ema10_pfcf_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_ema10_pe_price_portfolio = ema10_pe_ticker_totals.hvplot.line(title="Strategy EMA 10/100 on Price to Earnings (P/E) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

# EMA 30/100
plot_ema30_ps_price_portfolio = ema30_ps_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Sales (P/S) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_ema30_pfcf_price_portfolio = ema30_pfcf_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Free Cash Flow (P/FCF) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

plot_ema30_pe_price_portfolio = ema30_pe_ticker_totals.hvplot.line(title="Strategy EMA 30/100 on Price to Earnings (P/E) Ratio", 
                                  y = "portfolio_total",
                                  ylabel="Total Portfolio Value", 
                                  width=800, 
                                  height=400
).opts(yformatter='%.0f', show_grid=True, ylim=(350000, 800000))

In [44]:
# Display the SMA and EMA strategy total portfolio value plots
display(plot_sma10_ps_ratios)
display(plot_sma10_pfcf_ratios)
display(plot_sma10_pe_ratios)

display(plot_sma30_ps_ratios)
display(plot_sma30_pfcf_ratios)
display(plot_sma30_pe_ratios)

display(plot_ema10_ps_ratios)
display(plot_ema10_pfcf_ratios)
display(plot_ema10_pe_ratios)

display(plot_ema30_ps_ratios)
display(plot_ema30_pfcf_ratios)
display(plot_ema30_pe_ratios)

display(plot_sma10_ps_price_portfolio)
display(plot_sma10_pfcf_price_portfolio)
display(plot_sma10_pe_price_portfolio)

display(plot_sma30_ps_price_portfolio)
display(plot_sma30_pfcf_price_portfolio)
display(plot_sma30_pe_price_portfolio)

display(plot_ema10_ps_price_portfolio)
display(plot_ema10_pfcf_price_portfolio)
display(plot_ema10_pe_price_portfolio)

display(plot_ema30_ps_price_portfolio)
display(plot_ema30_pfcf_price_portfolio)
display(plot_ema30_pe_price_portfolio)


:NdOverlay   [Variable]
   :Curve   [Date]   (value)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

:NdOverlay   [Variable]
   :Curve   [Date]   (value)

...

:Curve   [Date]   (portfolio_total)

:Curve   [Date]   (portfolio_total)

:Curve   [Date]   (portfolio_total)

:Curve   [Date]   (portfolio_total)

:Curve   [Date]   (portfolio_total)

### Create a final portfolio comparison plot for the four indicator strategies

In [45]:
# Concat 'portfolio_total' columns from each DataFrame into a new DataFrame
all_portfolio_totals = pd.concat([
    sma10_ps_ticker_totals['portfolio_total'].rename('sma10_ps'),
    sma10_pfcf_ticker_totals['portfolio_total'].rename('sma10_pfcf'),
    sma10_pe_ticker_totals['portfolio_total'].rename('sma10_pe'),
    sma30_ps_ticker_totals['portfolio_total'].rename('sma30_ps'),
    sma30_pfcf_ticker_totals['portfolio_total'].rename('sma30_pfcf'),
    sma30_pe_ticker_totals['portfolio_total'].rename('sma30_pe'),
    ema10_ps_ticker_totals['portfolio_total'].rename('ema10_ps'),
    ema10_pfcf_ticker_totals['portfolio_total'].rename('ema10_pfcf'),
    ema10_pe_ticker_totals['portfolio_total'].rename('ema10_pe'),
    ema30_ps_ticker_totals['portfolio_total'].rename('ema30_ps'),
    ema30_pfcf_ticker_totals['portfolio_total'].rename('ema30_pfcf'),
    ema30_pe_ticker_totals['portfolio_total'].rename('ema30_pe')
], axis=1)

In [46]:
# Concat 'portfolio_total' columns from each DataFrame into a new DataFrame
all_portfolio_totals_final_day = all_portfolio_totals.tail(1)

In [47]:
# Transpose the DataFrame to turn columns into rows
transposed_df = all_portfolio_totals_final_day.T

# Sort the transposed DataFrame by its values in ascending order
sorted_df = transposed_df.sort_values(by=transposed_df.columns[0], ascending=True)

In [48]:
# Create a bar plot of the final portfolio value across different strategies and metrics
final_plot = sorted_df.hvplot.bar(
    title="Final Portfolio Total Value (2019-2024) - SMA/EMA from P/E, P/FCF, & P/S",
    xlabel="Strategies and Metrics",
    ylabel="Portfolio Total Value",
    width=800,
    height=400,
    rot=45  # Rotating the x-axis labels for better readability
).opts(yformatter='%.0f', show_grid=True, ylim=(400000, 750000), 
       fontsize = {"title": 14, "labels": 14, "xticks": 10, "yticks": 10})

In [49]:
# Plot the final comparison plot
final_plot

:Bars   [index]   (2024-04-01 00:00:00)